# VGGNet16 — Architektúra

Súbor: vggnet16.ipynb

Program: Hospodárska informatika

Vypracovala:  Veronika Motúzová 

Bakalárska práca: Metódy hlbokého učenia pre detekciu meteorov v astronomických snímkach

Vedúci bakalárskej práce: doc. Ing. Peter Butka, PhD.

Konzultant: Ing. Viera Maslej Krešňáková

architektúra podľa: https://medium.com/datadriveninvestor/five-powerful-cnn-architectures-b939c9ddd57b
podľa: https://medium.com/datadriveninvestor/five-powerful-cnn-architectures-b939c9ddd57b 
a https://gist.github.com/aaronpolhamus/034cfe4dd8e2cc26ccb24ad6616bf7b4

Import knižníc

In [2]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.metrics import categorical_crossentropy
from keras.layers import Conv2D, MaxPooling2D, Dropout, ZeroPadding2D
from keras.utils import np_utils, plot_model
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
import array
from sklearn.metrics import confusion_matrix,classification_report
import itertools 
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from keras.callbacks import ModelCheckpoint
from keras.models import load_model 

Načítanie dát

In [3]:
train = np.load('../data/1500_train.npz')
test = np.load('../data/1500_test.npz')

In [4]:
x_train = train['train_data'].reshape(train['train_data'].shape[0], 128, 128, 1)
x_test = test['test_data'].reshape(test['test_data'].shape[0], 128, 128, 1)

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [6]:
print(x_train.shape)
print(x_test.shape)

(16000, 128, 128, 1)
(16000, 128, 128, 1)


In [7]:
y_train = np_utils.to_categorical(train['train_targets'], 2)
y_test = np_utils.to_categorical(test['test_targets'], 2)

CNN VGGNet16

In [8]:
model = Sequential()

In [9]:
model.add(ZeroPadding2D((1,1),input_shape=(128, 128,1)))

In [10]:
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
#model.add(Conv2D(64, 3, 3, border_mode='valid', input_shape=(1, 128, 128),activation='relu'))

In [11]:
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))

In [12]:
model.add(MaxPooling2D((2,2), strides=(2,2)))

In [13]:
model.add(Dropout(0.25))

In [14]:
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))

In [15]:
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))

In [16]:
model.add(MaxPooling2D((2,2), strides=(2,2)))

In [17]:
model.add(Dropout(0.25))

In [18]:
model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))

In [19]:
model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))

In [20]:
model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))

In [21]:
model.add(MaxPooling2D((2,2), strides=(2,2)))

In [22]:
model.add(Dropout(0.25))

In [23]:
model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))

In [24]:
model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))

In [25]:
model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))

In [26]:
model.add(MaxPooling2D((2,2), strides=(2,2)))

In [27]:
model.add(Dropout(0.25))

In [28]:
model.add(Flatten())

In [29]:
model.add(Dense(2000, activation='relu'))

In [30]:
model.add(Dropout(0.5))

In [31]:
model.add(Dense(2000, activation='relu'))

In [32]:
model.add(Dropout(0.5))

In [33]:
model.add(Dense(2, activation='softmax'))

Adam Optimization

In [34]:
adam = Adam()

In [35]:
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

Aby sa uložil model s najlepším výsledkom:

In [36]:
saved_model = "vggnet16.hdf5"
checkpoint = ModelCheckpoint(saved_model, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [37]:
f = model.fit(x_train, y_train, epochs=15, batch_size=128, validation_data=(x_test, y_test), callbacks=[checkpoint])


Train on 16000 samples, validate on 16000 samples
Epoch 1/15
16000/16000 [==============================] - 2598s 162ms/step - loss: 0.6976 - accuracy: 0.5064 - val_loss: 0.6932 - val_accuracy: 0.5000

Epoch 00001: val_accuracy improved from -inf to 0.50000, saving model to vggnet16.hdf5
Epoch 2/15
16000/16000 [==============================] - 2555s 160ms/step - loss: 0.6932 - accuracy: 0.4999 - val_loss: 0.6932 - val_accuracy: 0.5000

Epoch 00002: val_accuracy did not improve from 0.50000
Epoch 3/15
16000/16000 [==============================] - 2555s 160ms/step - loss: 0.6933 - accuracy: 0.4953 - val_loss: 0.6932 - val_accuracy: 0.5000

Epoch 00003: val_accuracy did not improve from 0.50000
Epoch 4/15
16000/16000 [==============================] - 2489s 156ms/step - loss: 0.6932 - accuracy: 0.4998 - val_loss: 0.6932 - val_accuracy: 0.5000

Epoch 00004: val_accuracy did not improve from 0.50000
Epoch 5/15
16000/16000 [==============================] - 2487s 155ms/step - loss: 0.6931

Načítanie modelu

In [38]:
print("Loading model....")
model = load_model('vggnet16.hdf5')

Loading model....


Predikcia

In [39]:
y_pred = model.predict(x_test)
prediction_int = np.zeros_like(y_pred)
prediction_int[y_pred > 0.5] = 1

In [44]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.6931549309492111
Test accuracy: 0.5


Confusion Matrix

In [40]:
matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

In [41]:
print(matrix)
# TN / FN
# FP / TP

[[8000    0]
 [8000    0]]


Classification Report

In [42]:
print(classification_report(y_test, prediction_int))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67      8000
           1       0.00      0.00      0.00      8000

   micro avg       0.50      0.50      0.50     16000
   macro avg       0.25      0.50      0.33     16000
weighted avg       0.25      0.50      0.33     16000
 samples avg       0.50      0.50      0.50     16000



C:\Users\motuz\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 130, 130, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 126, 126, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 61, 61, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 59, 59, 128)      